Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [3]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import os
import time
import random as rd
import math

First reload the data we generated in _notmist.ipynb_.

In [4]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (350000, 28, 28) (350000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [5]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (350000, 784) (350000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [13]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [70]:
def run_model(num_epochs, batch_size,hidden_nodes, learning_rate, dropout_keep_prob, l2_penalty, \
                train_dataset,train_labels, path_to_save):
    num_steps = (train_labels.shape[0] * num_epochs) / batch_size
    epoch_log = np.full(num_steps, np.nan)
    loss_log = np.full(num_steps, np.nan)
    acc_log = np.full(num_steps, np.nan)
    val_acc_log = np.full(num_steps, np.nan)
    test_acc_log = np.full(num_steps, np.nan)
    print(num_steps)
    if not os.path.exists(path_to_save):
        os.makedirs(path_to_save)
        
    epoch_nb = -1
    graph = tf.Graph()
    # GRAPH
    with graph.as_default():
        # Input data. For the training data, we use a placeholder that will be fed
        # at run time with a training minibatch.

        tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)
        # Input parameters:
        tf_learning_rate = tf.placeholder(tf.float32)
        tf_dropout_keep_prob =  tf.placeholder(tf.float32)
        tf_l2_penalty =  tf.placeholder(tf.float32)

        # Variables.
        # 1 layer      
        hidden_weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_nodes], stddev=2 / math.sqrt(float(image_size * image_size))))
        hidden_biases = tf.Variable(tf.zeros([hidden_nodes]))
        hidden = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
        hidden_after_drop = tf.nn.dropout(hidden, tf_dropout_keep_prob)

        # 2 layer 
        hidden_weights_2 = tf.Variable(
        tf.truncated_normal([hidden_nodes, hidden_nodes], stddev=2 / math.sqrt(float(hidden_nodes))))
        hidden_biases_2 = tf.Variable(tf.zeros([hidden_nodes]))
        hidden_2 = tf.nn.relu(tf.matmul(hidden_after_drop, hidden_weights_2) + hidden_biases_2)
        hidden_after_drop_2 = tf.nn.dropout(hidden_2, tf_dropout_keep_prob)

        # 3 layer 
        hidden_weights_3 = tf.Variable(
        tf.truncated_normal([hidden_nodes, hidden_nodes], stddev=2 / math.sqrt(float(hidden_nodes))))
        hidden_biases_3 = tf.Variable(tf.zeros([hidden_nodes]))
        hidden_3 = tf.nn.relu(tf.matmul(hidden_after_drop_2, hidden_weights_3) + hidden_biases_3)
        hidden_after_drop_3 = tf.nn.dropout(hidden_3, tf_dropout_keep_prob)

        weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
        biases = tf.Variable(tf.zeros([num_labels]))

        # Training computation.
        logits = tf.matmul(hidden_after_drop_3, weights) + biases
        loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+tf_l2_penalty*tf.nn.l2_loss(weights)

        # Optimizer.
        optimizer = tf.train.AdamOptimizer(tf_learning_rate).minimize(loss)

        # Predictions for the training, validation, and test data.
        train_prediction = tf.nn.softmax(logits)
        valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(

                  tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases),

                                        hidden_weights_2) + hidden_biases_2),

                                        hidden_weights_3) + hidden_biases_3), weights) + biases)

        test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(
        tf.nn.relu(tf.matmul(
        tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases),hidden_weights_2) + hidden_biases_2),hidden_weights_3) + hidden_biases_3), weights) + biases)
 #starting new session     
    with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]

            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, 
                        tf_learning_rate : learning_rate, tf_dropout_keep_prob : dropout_keep_prob, 
                         tf_l2_penalty : l2_penalty}
            _, l, predictions = session.run(
                    [optimizer, loss, train_prediction], feed_dict=feed_dict)
            loss_log[step] = l
            epoch_log[step] = step * batch_size / float(train_labels.shape[0])
            if (step % 100 == 0):
                #print("Minibatch loss at step %d: %f" % (step, l))
                #print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                #print("Validation accuracy: %.1f%%" % accuracy( valid_prediction.eval(), valid_labels))
                
                val_acc_log[step] = accuracy(valid_prediction.eval(), valid_labels)
                acc_log[step] = accuracy(predictions, batch_labels)
                

        test_acc_log[step] = accuracy(test_prediction.eval(), test_labels)
        file_name = str(test_acc_log[step]) + '_ne.'+str(num_epochs)+'_bs.' + str(batch_size)+ '_lr.'+str(learning_rate) \
                    + '_d.' + str(dropout_keep_prob) +'_l2.'+ str(l2_penalty) + '_t'+'.csv'
        np.savetxt(os.path.join(path_to_save, file_name), np.vstack((epoch_log, loss_log, acc_log, val_acc_log, test_acc_log)).T, delimiter = ';', fmt='%.10f')
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Test if we get 100% accuracy on small training dataset:

In [65]:
run_model(300, 128,1024, 0.005, 1.0, 0, train_dataset[:256],train_labels[:256], '/notebooks/run1')

600
Minibatch loss at step 0: 29.219322
Minibatch accuracy: 8.6%
Validation accuracy: 16.4%
Minibatch loss at step 100: 0.000004
Minibatch accuracy: 100.0%
Validation accuracy: 61.0%
Minibatch loss at step 200: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 61.1%
Minibatch loss at step 300: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 61.1%
Minibatch loss at step 400: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 61.1%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 61.1%
Test accuracy: 67.3%


OK!

In [ ]:
num_epochs = 5
hidden_nodes = 1024
for i in range(50):
    t0 = time.time()
    batch_size = rd.choice([1,32,64,128])
    learning_rate = 10 ** rd.uniform(-6., 1.)
    dropout_keep_prob = rd.uniform(0,1)
    l2_penalty = 10 ** rd.uniform(-7., 1.)
    run_model(num_epochs, batch_size,hidden_nodes, learning_rate, dropout_keep_prob, l2_penalty, \
                train_dataset,train_labels, '/notebooks/run2')   
    t1 = time.time()
    print('Model',i,'- time:',(t1-t0)/60, 'min.')

array([[ 0.,  1.,  2.,  3.,  4.],
       [ 0.,  1.,  2.,  3.,  4.]])